# Project: Wrangling and Analyze Data

This project seeks to analyse data from WeRateDogs twitter archive. To successfully carry out this project, authorisation to query twitter api will be seeked. The project will be divided into the data wrangling section which includes; data gathering, assesment and cleaning; and data visualization

Install and import required packages

In [3]:
!pip install requests
!pip install tweepy

In [2]:
import pandas as pd
import json
import requests
import os
import tweepy
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
%matplotlib inline

ModuleNotFoundError: No module named 'requests'

# DATA WRANGLING

## Data Gathering
**All** three pieces of data for this project will be gathered and loaded in the notebook. The methods required to gather each data are different.

1. Directly download the WeRateDogs Twitter archive data (twitter_archive_enhanced.csv) provided by udacity

In [ ]:
enhanced_archive = pd.read_csv('twitter-archive-enhanced.csv')

2. Use the Requests library to download the tweet image prediction (image_predictions.tsv)

In [ ]:
url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
image_prediction = requests.get(url)
image_prediction

In [ ]:
if not os.path.exists('dataFolder'):
    os.makedirs('dataFolder')
    
with open (os.path.join('dataFolder', url.split('/')[-1]), mode = 'wb')as file:
    file.write(image_prediction.content)

img_prediction = pd.read_csv('dataFolder/image-predictions.tsv', sep = '\t')


3. Use the Tweepy library to query additional data via the Twitter API (tweet_json.txt)

In [ ]:
consumer_key = 'UzlBAp0jaXUGTHOAFv9pk2GzR'
consumer_secret = 'ZTuQ2YhfcWmUDQDi5xNCeyouJLYymTFtkiWEIkD41kmbqYTlmT'
access_token = '1302369654153805825-476CfXGLN0KrF4Vozvwn6G9IjHAdKV'
access_secret = 'zckl0isEr965Q1nZ7s36kjwPvBWxlHm1J4iX97AJCt52a'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth, wait_on_rate_limit=True)

In [ ]:
with open('tweet_json.txt', 'w') as file:
    for id in enhanced_archive['tweet_id']:
        try:
            tweet=api.get_status(id, tweet_mode='extended')
            data=tweet._json
            keys=data.key()
            value=data[key]
            dictionary = {keys:value}
            json.dump(dictionary, file)
            file.write('/n')
        except:
            pass 
print('completed')

In [ ]:
data_json = pd.read_json('tweet_json.txt', lines = True)  
data_json.head()

In [ ]:
with open ('tweet-json.txt') as file:
    lines = file.readlines()
    
data = []
for line in lines:
    data.append(json.loads(line))
    
data_json = pd.DataFrame(data)

## Assessing Data

All gathered dataset is to be assessed for quality and tidiness issues. These issues will be documented and used as guide for data cleaning to ensure a clean dataset. 

In [ ]:
img_prediction.sample(3)

In [ ]:
enhanced_archive.sample(10)

In [ ]:
data_json.sample(5)

In [ ]:
img_prediction.info()

In [ ]:
enhanced_archive.info()

In [ ]:
data_json.info()

In [ ]:
sum(img_prediction.duplicated())

In [ ]:
enhanced_archive.duplicated().sum()

In [ ]:
img_prediction.isnull().sum()

In [ ]:
enhanced_archive.isnull().sum()

In [ ]:
data_json.isnull().sum()

In [ ]:
img_prediction.shape

In [ ]:
enhanced_archive.shape

In [ ]:
data_json.shape

In [ ]:
img_prediction.describe()

In [ ]:
enhanced_archive.describe()

In [ ]:
data_json.describe()

### Quality issues
1. wrong datatype(tweet_id, timestamp, retweeted status_id)


2. consistency issues across tables (tweet_id and id)


3. different number of tweetid entries among three data frames


4. multiple p_conf; p2_conf and p3_conf has outliers; wide difference between min and 25%.


5. false value under p_dog column


6. **enhanced_archive df**: unnecessary columns(expanded urls, inreply to  status/userid, retweeted_status_user_id, retweeted_status_timestamp)


7. **enhanced_archive df**: unwanted information (retweeted non-null rows/entries)


8. **enhanced_archive df**: outliers in ratings numerator and denominator


9. **data_jason dataframe**: column repetition(id_str/id, in_reply_to_status_id/in_reply_to_status_id_str, in_reply_to_user_id /in_reply_to_user_id_str)


10. **data_jason df**: unnecessary columns(possibly_sensitive,possibly_sensitive_appealable,lang,retweeted_status,quoted_status_id,quoted_status_id_str,quoted_status, created at, entities, extended_entities, truncated, display_text_range, source, favourited, retweeted) 


11. **data_json df**: retweeted status not null, quoted status_id not null


12. null columns: geo, coordinates, place , contributors   



### Tidiness issues
1. timestamp column on enhanced_archive dataframe contains both date and time.


2. information on dog stage are in different columns. This defies the law of tidiness


3. data on different tables

## Cleaning Data
In this section,**all** of the issues documented while assessing will be cleaned. 

copies of original dataset will be made to ensure that the original dataset is not tampered with.

In [ ]:
# Make copies of original pieces of data
enhanced_clean = enhanced_archive.copy()
image_clean = img_prediction.copy()
json_clean = data_json.copy()

### wrong datatype (tweet_id, retweet_id, timestamp)

#### Define:

 use astype() to correct datatypes

#### Code

In [ ]:
enhanced_clean['tweet_id'] = enhanced_clean.tweet_id.astype(str)
image_clean['tweet_id'] = image_clean.tweet_id.astype(str)
enhanced_clean['retweeted_status_id'] = enhanced_clean.retweeted_status_id.astype(str)
enhanced_clean['timestamp'] = enhanced_clean.timestamp.astype('datetime64')

#### Test

In [ ]:
enhanced_clean.dtypes

In [ ]:
image_clean.dtypes

### use one prediction algorithm

#### Define

since p1 had higest p1_conf max, drop p2, p2_conf, p2_dog, p3, p3_conf and p3_dog

#### Code

In [ ]:
image_clean.drop(['p2','p2_conf','p2_dog','p3','p3_conf','p3_dog'], axis=1, inplace=True)

#### Test

In [ ]:
image_clean.head(2)


In [ ]:
image_clean.p1_dog.value_counts()

###### DEFINE

drop p1_dog = false

###### CODE

In [ ]:
image_clean=image_clean.query('p1_dog==True')

###### TEST

In [ ]:
image_clean.p1_dog.value_counts()

### Dog stage information in multiple columns

###### DEFINE

extract dog_stage information from text and fill dog_stage column using the information. drop 'doggo', 'floofer', 'pupper', 'puppo' columns

###### CODE


In [ ]:
text = list(enhanced_clean.text)
stg = []
a = 'doggo'
b = 'floofer'
c = 'pupper'
d = 'puppo'
for line in text:
    if a in line:
        stg.append(a)
    elif b in line:
        stg.append(b)
    elif c in line:
        stg.append(c)
    elif d in line:
        stg.append(d)
    else:
        stg.append('None')
            
        
len(stg ) 

       
enhanced_clean['dog_stage'] = stg   

In [ ]:
enhanced_clean.drop(['doggo', 'floofer', 'pupper', 'puppo'], axis = 1, inplace = True)

##### test

In [ ]:
enhanced_clean.head(3)

### Unnecessary colums and not-null retweeted_status_id

###### DEFINE

Drop unnecessary columns(expanded urls, inreply to  status/userid, retweeted_status_user_id, retweeted_status_timestamp, source,text) from enhanced_clean and  unwanted information (retweeted non-null rows/entries)

###### CODE

In [ ]:
enhanced_clean.drop(['expanded_urls', 'in_reply_to_status_id', 'in_reply_to_user_id', 'retweeted_status_user_id', 'retweeted_status_timestamp', 'source'], axis = 1, inplace = True)

In [ ]:
enhanced_clean= enhanced_clean.query('retweeted_status_id== "nan"')

###### TEST

In [ ]:
enhanced_clean.head(3)

In [ ]:
enhanced_clean['retweeted_status_id'].value_counts()

###### CODE

In [ ]:
enhanced_clean.drop('retweeted_status_id', axis=1, inplace=True)

###### TEST

In [ ]:
enhanced_clean.head(3)

### Possible rating extraction error

###### DEFINE

extract rating num and denominator from text and drop text column

###### CODE

In [ ]:
enhanced_clean['rating_numerator']=enhanced_clean.text.str.extract('(\d\d)', expand = True)
enhanced_clean['rating_denominator']=enhanced_clean.text.str.extract('((?<=/)\d\d)', expand = True)

In [ ]:
enhanced_clean.drop('text', axis=1, inplace=True)

###### TEST

In [ ]:
enhanced_archive.describe()

### Outliers 

###### DEFINE

Drop rating_numerator greater than 14 and ratung_denominator greater than 10

###### CODE

In [ ]:
enhanced_archive=enhanced_archive.query('rating_numerator<=14')

In [ ]:
enhanced_archive=enhanced_archive.query('rating_denominator==10')

###### TEST

In [ ]:
enhanced_archive.describe()

### Unnecessary columns and null columns

###### DEFINE

Drop unnecessary columns(possibly_sensitive,possibly_sensitive_appealable,lang,in_reply_to_screen_name, quoted_status, created at, entities, extended_entities, truncated, display_text_range, source, favourited, retweeted, user)

duplicated columns ('in_reply_to_status_id_str', 'in_reply_to_user_id', 'in_reply_to_user_id_str', in_reply_to_status_id, id_str, quoted_status_id,quoted_status_id_str,) 

and null columns: geo, coordinates, place , contributors on json_clean dataframe

###### CODE

In [ ]:
json_clean.drop(['possibly_sensitive', 'possibly_sensitive_appealable', 'lang', 'quoted_status_id', 'quoted_status_id_str', 'quoted_status', 'created_at', 'entities', 'extended_entities', 'truncated', 'display_text_range', 'source', 'favorited', 'retweeted', 'geo', 'coordinates', 'place', 'contributors', 'in_reply_to_status_id_str', 'in_reply_to_user_id', 'in_reply_to_user_id_str', 'in_reply_to_screen_name', 'in_reply_to_status_id', 'user', 'id_str'], axis = 1, inplace = True)

###### TEST

In [ ]:
json_clean.head(3)

### unneeded rows

###### DEFINE

Query is_quote_status not equal to NAN and drop the column

###### CODE


In [ ]:
json_clean=json_clean.query('is_quote_status== False')

###### TEST

In [ ]:
json_clean.is_quote_status.value_counts()

###### CODE

In [ ]:
json_clean.drop('is_quote_status', axis=1, inplace=True)

###### TEST

In [ ]:
json_clean.head(3)

### Column name inconsistency

###### DEFINE

Change id name to match other tables and change its datatype

###### CODE

In [ ]:
json_clean.rename(columns = {'id':'tweet_id'}, inplace = True)
json_clean['tweet_id'] = json_clean['tweet_id'].astype(str)

###### TEST

In [ ]:
json_clean.info()

### Retweets not needed

###### DEFINE

drop retweeted_status not NaN and drop the column

###### CODE

In [ ]:
json_clean = json_clean[json_clean.retweeted_status.isnull()]

###### TEST

In [ ]:
json_clean.sample(50)

###### CODE

In [ ]:
json_clean.drop('retweeted_status', axis=1, inplace=True)

###### TEST

In [ ]:
json_clean.head()

### Data in different table

###### DEFINE

merge tataframes and join based on common tweet_id

###### CODE

In [ ]:
twitter_archive_master=pd.merge(pd.merge(json_clean, image_clean, on=['tweet_id'], how='inner'),enhanced_clean, on=['tweet_id'], how='inner')

###### TEST

In [ ]:
twitter_archive_master.head(3)

## Storing Data
The gathered, assessed, and cleaned master dataset will be saved to a CSV file named "twitter_archive_master.csv".

In [ ]:
twitter_archive_master.to_csv('twitter_archive_master.csv', index=False)

## Analyzing and Visualizing Data

The cleaned dataset will be analyzed further and visualized to draw insight

In [ ]:
retweets=twitter_archive_master.groupby('p1').retweet_count.mean().nlargest(n=5)
retweets

In [ ]:
likes=twitter_archive_master.groupby('p1').favorite_count.mean().nlargest(n=5)
likes

In [ ]:
rating=twitter_archive_master.groupby('p1').rating_numerator.mean().nlargest(n=5)
rating

In [ ]:
twitter_archive_master.p1.value_counts()

In [ ]:
dogs=twitter_archive_master.groupby('dog_stage').mean()
dogs

In [ ]:
year =twitter_archive_master.timestamp.dt.year
time=twitter_archive_master.groupby(by=year).sum()
time
#twitter_archive_master.timestamp.dt.year

In [ ]:
time['sum']=time.retweet_count+time.favorite_count
time['year']= ('2015','2016','2017')
time

### Insights:
1. Afghan_hound and Saluki, Great_Pyrenees had top tweet engagement since both were among top liked and retweeted


2. golden_retriever, Labrador_retriever, Pembroke, Chihuahua, pug are top 5 dogs by rating


3. year 2017 had highest engagement. this shows growth for the page

### Visualization

In [ ]:
time.plot(x='year', y='sum', kind='line', figsize=(10,9))
plt.title('Activity trendline over three years')
plt.ylabel('ACTIVITIES')
plt.xlabel('YEAR')

In [ ]:
dogs_retweet = dogs.retweet_count
label=dogs.index
colour = sb.color_palette('pastel');
plt.pie(data= dogs, x= dogs_retweet, labels=label, color = colour, counterclock=False, startangle=90, normalize=True, autopct='%.2f');
plt.title('Best dog stage by retweeet count');

In [ ]:
dogs_likes=dogs.favorite_count
label=dogs.index
plt.pie(data= dogs, x= dogs_likes, labels=label, color = colour, counterclock=False, startangle=90, normalize=True, autopct='%.2f');
plt.title('Most loved dog_stage');